In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os

In [2]:
daysLimit = int(input('How many days back in history?'))

In [50]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [3]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO BORROWS FIRST

In [39]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [40]:
oldSiloBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmBorrows.csv'
try:
    oldSiloBorrowsDb = pd.read_csv (siloFileName)
except:
    oldSiloBorrowsDb = pd.DataFrame()
oldSiloBorrowsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0xf8e0d6896a0422e8f3cb994a0d40ba911a48426331e0...,16717823,1677480203,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,30000.00,0.997992,30025.436160,2023-02-27 06:43:23,2023-02-27
1,borrow,0x164ddb76e787d0d47799d9f4d2fa4c5dbbe3cec645bc...,16716960,1677469739,0xbe45fb24bd0f1f348cd21fd8cf3d1e94afc9ee58,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,4500.00,0.997992,4485.559896,2023-02-27 03:48:59,2023-02-27
2,borrow,0x220922454f571a334ef370d9122aaec2f16eb4f2b137...,16701840,1677286115,0xa0fea50194bd39d5b3da8a795b36db75313b1936,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,2050.00,0.997992,2048.472075,2023-02-25 00:48:35,2023-02-25
3,borrow,0xb3ff0296b2a31e48cf7aa26a5dbe4ffd8cf504579227...,16699182,1677253703,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,50000.00,0.997992,49977.205520,2023-02-24 15:48:23,2023-02-24
4,borrow,0x52f5ba92b7e35bde0ac2d595b6f55c6fcf312c7b6292...,16695694,1677211127,0x804157889ac56a4e50278610c67960c1605a3264,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,700.00,0.997992,702.225601,2023-02-24 03:58:47,2023-02-24
5,borrow,0xd1e0d42b27bc7304ce2aa914d63058fbf5be6709aa77...,16686330,1677097475,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,3.25,1618.907560,5270.606820,2023-02-22 20:24:35,2023-02-22
6,borrow,0x5946a769e10307668b4b60aa3b36d5189916cc02fc23...,16684007,1677069179,0x5656cb4721c21e1f5dcbe03db9026ac0203d6e4f,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,4520.00,0.997992,4510.003371,2023-02-22 12:32:59,2023-02-22
7,borrow,0x7a9527e13f7627c09cc9bdcc43d1aa6a7babf0c087b8...,16681973,1677044447,0x7aaee144a14ec3ba0e468c9dcf4a89fdb62c5aa6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,16500.00,0.997992,16472.437498,2023-02-22 05:40:47,2023-02-22
8,borrow,0x84778306b3f662f7d98e431591f302d924d143c3c0b2...,16678471,1677001823,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,6540.00,0.997992,6588.024198,2023-02-21 17:50:23,2023-02-21
9,borrow,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,16651434,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,60500.00,0.997992,59947.550425,2023-02-17 22:39:23,2023-02-17


In [41]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldSiloBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

1677480198

In [42]:
# 0xc413dd03555f3eb29d834b482d386b2999dc2eb0

In [43]:
##this captures the ENTIRE list of people who borrowed stablecoins against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  6  - latest recordID:  0xf8e0d6896a0422e8f3cb994a0d40ba911a48426331e0182f5e1220778e1472cd   0xf8e0d6896a0422e8f3cb994a0d40ba911a48426331e0182f5e1220778e1472cd-137  - borrows DB length:  6  - latest record from:  2023-02-26 22:43:23 0 days ago
Done. Total Records collected:  6


In [44]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])

siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0xf8e0d6896a0422e8f3cb994a0d40ba911a48426331e0...,16717823,1677480203,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,30000.0,0.997992,30025.43616,2023-02-27 06:43:23,2023-02-27


In [45]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmBorrows_piped.csv file saved, 1  records


In [46]:
totalBorrowedUSDSilo = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of $',round(totalBorrowedUSDSilo,2),  'has been borrowed in different stablecoins over time over at Silo')

A total of $ 30025.44 has been borrowed in different stablecoins over time over at Silo


In [47]:
gOHMBorrowedSilo = db.query("Select distinct "
                           "transaction_hash "
                           ",transaction_timestamp "
                           ", dateTime "
                           ",transaction_account_id "
                           ", transaction_asset_symbol "
                           ", transactionReadableAmount "
                           ", transaction_amountUSD "
                           "from siloBorrowsDb where lower(transaction_asset_symbol) = 'gohm'").df()

gOHMBorrowedSilo.head(10)

,transaction_hash,transaction_timestamp,dateTime,transaction_account_id,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD


In [48]:
totalSiloBorrowedgOHM = sum(gOHMBorrowedSilo['transactionReadableAmount'])
totalSiloBorrowedgOHMinUSD = sum(gOHMBorrowedSilo['transaction_amountUSD'])
print('A total of ',round(totalSiloBorrowedgOHM,2),  'OHM has been borrowed in different stablecoins over time in Silo, roughly USD$',totalSiloBorrowedgOHMinUSD)

A total of  0 OHM has been borrowed in different stablecoins over time in Silo, roughly USD$ 0


In [15]:
path =file+'/ohmBorrowedSilo.csv'
gOHMBorrowedSilo.to_csv(path, index = False)

path =file+'/ohmBorrowedSilo_piped.csv'
gOHMBorrowedSilo.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(gOHMBorrowedSilo),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/ohmBorrowedSilo_piped.csv file saved, 0  records


********************************************************************
********************************************************************
SILO COLLATERAL DEPOSITS

In [16]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2838.193286,138354.126796,2023-02-17 22:34:23,2023-02-17
1,deposit,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,50.749992,2838.193286,145833.530065,2023-02-16 13:58:59,2023-02-16
2,deposit,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,16627499,1676383883,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.180346,2838.193286,490.375269,2023-02-14 14:11:23,2023-02-14
3,deposit,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,16627469,1676383523,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.663108,2838.193286,1803.040025,2023-02-14 14:05:23,2023-02-14
4,deposit,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.892669,2838.193286,2431.527276,2023-02-13 23:18:59,2023-02-13
5,deposit,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.087799,2838.193286,2963.036715,2023-02-13 23:14:47,2023-02-13
6,deposit,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,16619969,1676292887,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.812154,2838.193286,4846.273689,2023-02-13 12:54:47,2023-02-13
7,deposit,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2838.193286,363.447391,2023-02-10 22:58:11,2023-02-10
8,deposit,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,9.990000,2838.193286,26839.810526,2023-02-09 10:34:59,2023-02-09
9,deposit,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,16589368,1675923587,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,119.000000,2838.193286,315110.337935,2023-02-09 06:19:47,2023-02-09


In [17]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositsDb['transaction_timestamp'].max()))-1
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)
maxSiloDepositsTimestamp

1676673262

In [18]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if siloDepositsDbLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  54  - latest recordID:  0x29643d66a812414ade1fe477e9cbb4f138a883192f7db053d91023dce7221ac3   0x29643d66a812414ade1fe477e9cbb4f138a883192f7db053d91023dce7221ac3-195  - deposits DB length:  54  - latest record from:  2023-02-26 22:38:11 0 days ago
Done. Total Records collected:  54


In [19]:
siloDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
1,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,29.269396,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
2,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.858009,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
3,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
4,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,22.493314,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
5,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,4.992150,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2706.113816,20449883374896217620,55829.594023,deposit
6,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,16716950,1677469619,0xbe45fb24bd0f1f348cd21fd8cf3d1e94afc9ee58,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,3624736748660984613,9895.140234,deposit
7,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,16716950,1677469619,0xbe45fb24bd0f1f348cd21fd8cf3d1e94afc9ee58,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,29.269396,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,3624736748660984613,9895.140234,deposit
8,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,16716950,1677469619,0xbe45fb24bd0f1f348cd21fd8cf3d1e94afc9ee58,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.858009,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d

In [20]:
siloDepositsDb['event'] ='deposit'
siloDepositsDb['transactionReadableAmount'] = siloDepositsDb['transaction_amount']/(10**(siloDepositsDb['transaction_asset_decimals']))
siloDepositsDb['transactionReadableAmount']=siloDepositsDb['transactionReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['transaction_timestamp'],unit='s'))


#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(1000)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0x29643d66a812414ade1fe477e9cbb4f138a883192f7d...,16717797,1677479891,0x5c9d09716404556646b0b4567cb4621c18581f94,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,20.449883,2706.113816,55829.594023,2023-02-27 06:38:11,2023-02-27
6,deposit,0xbf0ed7c716a01e4be252250119bd41cdf89f65597040...,16716950,1677469619,0xbe45fb24bd0f1f348cd21fd8cf3d1e94afc9ee58,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,3.624737,2706.113816,9895.140234,2023-02-27 03:46:59,2023-02-27
12,deposit,0x99a647b2e80fef593cdd66b3c6abb8e362c49aee85bf...,16701789,1677285503,0xa0fea50194bd39d5b3da8a795b36db75313b1936,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,2.013776,2706.113816,5483.214093,2023-02-25 00:38:23,2023-02-25
18,deposit,0xe7bbf9cfb082dbb38c82a00958c376852afcabf6a5e7...,16699172,1677253583,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,54.048930,2706.113816,144122.371468,2023-02-24 15:46:23,2023-02-24
24,deposit,0x023c3294dcbafb52f4a907af98ea8954c6c2d8d88c48...,16695675,1677210899,0x804157889ac56a4e50278610c67960c1605a3264,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.374146,2706.113816,1023.449957,2023-02-24 03:54:59,2023-02-24
30,deposit,0x0558a1b5042de6895f694f913f5006a7080a1671de70...,16683975,1677068795,0x5656cb4721c21e1f5dcbe03db9026ac0203d6e4f,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,3.138941,2706.113816,8614.397300,2023-02-22 12:26:35,2023-02-22
36,deposit,0x47a01a4e9b92bd82f154203b765cd526234e741c5cda...,16681931,1677043943,0x7aaee144a14ec3ba0e468c9dcf4a89fdb62c5aa6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,9.937799,2706.113816,27352.209304,2023-02-22 05:32:23,2023-02-22
42,deposit,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,5.621351,2706.113816,15493.665575,2023-02-21 17:49:35,2023-02-21
48,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2706.113816,138354.126796,2023-02-17 22:34:23,2023-02-17


In [21]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmDeposits_piped.csv file saved, 9  records


In [22]:
totalDeposited = sum(siloDepositEventsDbTimedClean['transactionReadableAmount'])
totalUSDDeposited = sum(siloDepositEventsDbTimedClean['transaction_amountUSD'])
print('There has been a total of',round(totalDeposited,2),  'gOHM deposited cumulatively as collateral  to date, about USD$',totalUSDDeposited)

There has been a total of 148.13 gOHM deposited cumulatively as collateral  to date, about USD$ 406168.1687487883


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [23]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,withdraw,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,25.000039,2838.193286,71148.084004,2023-02-15 15:27:47,2023-02-15
1,withdraw,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,64.393035,2838.193286,183257.358356,2023-02-15 14:49:59,2023-02-15
2,withdraw,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,36.000000,2838.193286,100991.276713,2023-02-14 16:23:59,2023-02-14
3,withdraw,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,16627341,1676381987,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.883213,2838.193286,2401.521741,2023-02-14 13:39:47,2023-02-14
4,withdraw,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,16601676,1676072123,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2838.193286,363.447391,2023-02-10 23:35:23,2023-02-10
5,withdraw,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,16589407,1675924055,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,19.607121,2838.193286,51919.381911,2023-02-09 06:27:35,2023-02-09
6,withdraw,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,16574012,1675737947,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.194798,2838.193286,510.055435,2023-02-07 02:45:47,2023-02-07
7,withdraw,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,16483146,1674641375,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,20.000000,2838.193286,51345.559733,2023-01-25 10:09:35,2023-01-25
8,withdraw,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,16366146,1673230019,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,2.252766,2838.193286,5812.081890,2023-01-09 02:06:59,2023-01-09
9,withdraw,0x7c24d04318da39f08c7ba4f8d413a487133ce1b1f117...,16242703,1671741731,0x96035b45bc0d22f0ed11283a91145fb001c48370,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,13.520668,2838.193286,34629.263098,2022-12-22 20:42:11,2022-12-22


In [24]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawsDb['transaction_timestamp'].max()))-1
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

1676474866

In [25]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  42  - latest recordID:  0xab801d85bf2eda4b0e198678991805292ea062413e39e1acaece27c107755deb   0xab801d85bf2eda4b0e198678991805292ea062413e39e1acaece27c107755deb-222  - withdraws DB length:  42  - latest record from:  2023-02-26 02:44:47 1 days ago
Done. Total Records collected:  42


In [26]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
0,0xab801d85bf2eda4b0e198678991805292ea062413e39...,0xab801d85bf2eda4b0e198678991805292ea062413e39...,16711899,1677408287,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,8000012422410633960,21812.764574,withdraw,8.000012,2023-02-26 10:44:47
1,0xab801d85bf2eda4b0e198678991805292ea062413e39...,0xab801d85bf2eda4b0e198678991805292ea062413e39...,16711899,1677408287,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,29.269396,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,8000012422410633960,21812.764574,withdraw,8.000012,2023-02-26 10:44:47
2,0xab801d85bf2eda4b0e198678991805292ea062413e39...,0xab801d85bf2eda4b0e198678991805292ea062413e39...,16711899,1677408287,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.858009,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2706.113816,8000012422410633960,21812.764574,withdraw,8.000012,2023-02-26 10:44:47
6,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,16711629,1677405035,0x44504ba8d43b30e882503948654e247b928bcc7f,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,500000776400664622,1363.297786,withdraw,0.500001,2023-02-26 09:50:35
7,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,16711629,1677405035,0x44504ba8d43b30e882503948654e247b928bcc7f,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,29.269396,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,500000776400664622,1363.297786,withdraw,0.500001,2023-02-26 09:50:35
8,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,0xb4d1f10a88462b4f8e0c10ce406c209b909d6661cc14...,16711629,1677405035,0x44504ba8d43b30e882503948654e247b928bcc7f,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.858009,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2706.113816,500000776400664622,1363.297786,withdraw,0.500001,2023-02-26 09:50:35
12,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,16706550,1677343367,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2706.113816,9990015512485279153,27306.929822,withdraw,9.990016,2023-02-25 16:42:47
13,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,16706550,1677343367,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,29.269396,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2706.113816,9990015512485279153,27306.929822,withdraw,9.990016,2023-02-25 16:42:47
14,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,0x8d78cb38f2bd316df301e67a0a64272156228c708266...,1670655

In [27]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmWithdraws_piped.csv file saved, 5  records


In [28]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 70.7 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 194175.62


In [29]:
siloCollateralBalance = totalDeposited -(totalWithdrawed)
print(totalDeposited, ' - ',totalWithdrawed,'=', siloCollateralBalance, 'currently deposited as collateral in Silo' )

148.13176131732266  -  70.69580280753544 = 77.43595850978721 currently deposited as collateral in Silo


********************************************************************
********************************************************************
VESTA BORROWS - Their API is poo RN

In [30]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/shinitakunai/vestafinance-v1_2')

In [31]:
oldVestaBorrowsDb = pd.DataFrame()
siloFileName = file+'/vesta_borrow_events.csv'
try:
    oldVestaBorrowsDb = pd.read_csv (siloFileName)
except:
    oldVestaBorrowsDb = pd.DataFrame()
oldVestaBorrowsDb.head(100)

,borrows_id,borrows_hash,borrows_amount,borrows_amountUSD,borrows_timestamp,borrows_account_id,borrows_market_id,borrows_market_name,borrows_market_rates_rate,borrows_asset_id,borrows_asset_name,borrows_asset_symbol,borrows_asset_decimals,borrows_asset_lastPriceUSD,borrows_asset_lastPriceBlockNumber,event,dateTime
0,borrow-0xe78e7cf33f3c3dcf9d167c79992bd499f5e26...,0xe78e7cf33f3c3dcf9d167c79992bd499f5e261aa781a...,576298224974605960408026,576298.224975,1659911497,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-08-07 22:31:37
1,borrow-0x93e6a1fc4275b1e3229824808cfbd67b1a95b...,0x93e6a1fc4275b1e3229824808cfbd67b1a95b70da5fc...,516405000007276459170000,516405.000007,1661574660,0x15bae977db3b62ceb5a67921fae589fd80624e8d,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-08-27 04:31:00
2,borrow-0x9651ee75040616476d15e135efd447ea2b61f...,0x9651ee75040616476d15e135efd447ea2b61f1c4a1a4...,508522687608107465832660,508522.687608,1661405272,0xea17e11fa9d3d772eee6f34a03f0ef44db395720,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-08-25 05:27:52
3,borrow-0x6d78d97f945a157ac5bddb6e4fc1f082a9026...,0x6d78d97f945a157ac5bddb6e4fc1f082a9026a1d4877...,407615492208601338652456,407615.492209,1669999334,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-12-02 16:42:14
4,borrow-0x7ebbec623197b45efee3ee5cea51c5570ced5...,0x7ebbec623197b45efee3ee5cea51c5570ced507e35b7...,398065447708496993829189,398065.447708,1670001231,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-12-02 17:13:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,borrow-0x4971939a1fa82aac80d47cb84bf22906a6a6c...,0x4971939a1fa82aac80d47cb84bf22906a6a6c1dc1670...,61341003306669234265579,61341.003307,1670604649,0x6ae68eb2fa416fe091803fde014bbf7475468979,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-12-09 16:50:49
96,borrow-0x0d23c88566be4ab100a116a2ec18098d2d232...,0x0d23c88566be4ab100a116a2ec18098d2d2326206389...,61087922925193565302356,61087.922925,1658654695,0x35e8557f4236bd3f72d0a1a645d06791b0d3bf28,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-07-24 09:24:55
97,borrow-0x3dae6c0f28205ccd8229d7e19e3c336137449...,0x3dae6c0f28205ccd8229d7e19e3c336137449597a9e3...,60780009619823879220000,60780.009620,1659334359,0xcf8e9a44c47461cd099f73fe6b4c2bc609a14e2a,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-08-01 06:12:39
98,borrow-0x37ebd9187b895f2d610276994cbe5c7a9489d...,0x37ebd9187b895f2d610276994cbe5c7a9489dcd18329...,60780000000000000000000,60780.000000,1651452956,0x6dd91bdab368282dc4ea4f4befc831b78a7c38c0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,Governance OHM,0.0,0x64343594ab9b56e99087bfa6f2335db24c2d1f17,Vesta Stable,VST,18,0.979297,63919783,borrow,2022-05-02 00:55:56


In [32]:
maxVestaBorrowTimestamp = 0
try:
    maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
    maxVestaTroveChangeId = int(maxVestaTroveChangeId)
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
        )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])
except:
    maxVestaTroveChangeId = 0
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
               #,'id_gte': maxVestaTroveChangeId
         }
    )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])

vestaBorrowslist.head(10)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name
0,99981,0xf935941fd9f2518fa2a5f5f8d29dfa38c92fccb25b75...,34713629,-0.100000,1667397544,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
1,99820,0x467b6aa4a3f3d45e9316a143c45bf91c83ed1f662a4b...,34701032,-0.010000,1667393421,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
2,9972,0xfd781bb00f9e6d9599ce7aebda95979bfb902d3d73d2...,11845437,-2848.848834,1652313237,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
3,99669,0xe104d6896bf784009bd9aa410f077f9c0a25154ab628...,34691865,-0.100000,1667390188,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
4,9962,0x2fde74916c909648c83e2725fecdc902181371a58c1f...,11842359,-700.000000,1652309981,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
5,9961,0x1ebcd76a95f023492880bf7f7fbf3183e2197dd266a2...,11841593,-156.000000,1652309386,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
6,99531,0xc4558983947cedeace28d691cf9a70d871256ccd9021...,34681251,-0.020000,1667386332,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
7,9924,0x6a478fd975875045387ad810583e8270a25530599a6d...,11835231,-4000.003949,1652304099,0x6688b44ff874e246c441397d60fc169d4972a1ab,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
8,99043,0x24f693681f80f12946d463995c424c41f37a01dbd227...,34647273,-31300.004280,1667374762,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
9,9896,0xdf29dc3054d53c0c8681ba1e5a9ab72d0dafbcf77de4...,11831764,-222.000000,1652301736,0x7ff15d6cf1a7bf0efa7435543cf086835fc80d53,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM


vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
    )
vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.transaction.id
        ,vestaBorrows.transaction.blockNumber
        ,vestaBorrows.debtChange
        ,vestaBorrows.transaction.timestamp
        ,vestaBorrows.trove.owner.id
        ,vestaBorrows.trove.asset.id
        ,vestaBorrows.trove.asset.name
    ])

vestaBorrowslist.head(10)

In [33]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    try:
        maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
        maxVestaTroveChangeId = int(maxVestaTroveChangeId)
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                    ,'id_gte': maxVestaTroveChangeId
             }
            )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])
    except:
        maxVestaTroveChangeId = 0
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                   #,'id_gte': maxVestaTroveChangeId
             }
        )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0x21da8f0687a20a391e7e08c8af41efd288af11da388165d1fe8423ff8468ed59   9966  - vestaBorrows DB length:  1000  - latest record from:  2022-05-11 16:41:54 292 days ago
iterations:  1 Lines skipped:  1000 records collected:  447  - latest recordID:  0xf29d71ee0846f86ac5b6ae454da2078f3061e0278c66a491a64de4dedde05bf3   991  - vestaBorrows DB length:  1447  - latest record from:  2022-02-09 04:29:20 383 days ago
Done. Total Records collected:  1447


In [34]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(troveChanges_transaction_timestamp::int) dateTime "
               "From vestaBorrowsDb "
                               "order by troveChanges_transaction_timestamp desc"
               ).df()

vestaBorrowsDbTimed.head(1000)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name,event,dateTime
0,208699,0x0262303378c58366472df669090149e9fc7724ee9adb...,65178741,300.000000,1677525996,0x18fee6e511646b8ea2e4964277574f637bc03f3b,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-27 19:26:36
1,208622,0x55df82816c96cd12af9203c47081a944c89214ea5d3c...,65112766,0.453766,1677509290,0x57113e17edf278e15e58f047a7aa64dd826f1347,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-27 14:48:10
2,208607,0xfce0fc3cb35b1f04d847e4931eac3b116c70dd380c6f...,65079018,14999.997387,1677500902,0xded5d8c6b271a48f3b9ba20e8aa761a6baa2fac9,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-27 12:28:22
3,208574,0x9d0ce7a1a0fb412b468d3b609f4646020721ed89501b...,64918656,7022.916997,1677459782,0x0fd84d7cb911728737556684050782b298f70f0f,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-27 01:03:02
4,208528,0x42ff27c42a22e3a79c74807f016ab9f79340a653bcaa...,64712141,147.219475,1677404640,0x57113e17edf278e15e58f047a7aa64dd826f1347,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-26 09:44:00
...,...,...,...,...,...,...,...,...,...,...
995,7945,0xe6e412a75c883dc4f29387b7ce4a9dd9a93daffd3330...,10348558,28053.843750,1650656395,0x445e2f0b03f5d61a141913819b18c83aa69f29e5,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-04-22 19:39:55
996,7944,0x6f55c86ca5c272501eaebe2480cec8d3176b4f7e2b25...,10348401,1812.434010,1650656175,0x0c3db603a4045d0f6e8167a9bf0276a4697573a7,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-04-22 19:36:15
997,7943,0x97eb09633d60afb258b2c87a5a5e694caabe3d0629ca...,10348299,4858.781098,1650656030,0x344282db40f3903deefcd63e7cd665afdec94616,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-04-22 19:33:50
998,7936,0xe679e8f5d0c183e9755fee718789e9bb8bd8967f050c...,10338698,199.130273,1650643308,0x16d81b2dfd8d6650b2013ebed328588e7c26da24,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-04-22 16:01:48


In [35]:
vestaBorrowsDbTimed = pd.concat([oldVestaBorrowsDb,vestaBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
vestaBorrowsDbTimed.drop_duplicates()
vestaBorrowsDbTimed = db.query("select "
               "    * "
               "From vestaBorrowsDbTimed "
                         "group by 1,2,3,4,5,6,7,8,9,10 "
                     # "order by borrows_amountUSD desc "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

RuntimeError: Binder Error: column "borrows_asset_name" must appear in the GROUP BY clause or be used in an aggregate function

In [ ]:
path =file+'/vesta_borrow_events.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(vestaBorrowsDbTimed),' records')

************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [ ]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
#frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [ ]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_actions.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

In [ ]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_borrows.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

In [ ]:
oldFraxDeposits = pd.DataFrame()
fraxFileName = file+'/frax_deposits.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

In [ ]:
oldFraxWithdraws = pd.DataFrame()
fraxFileName = file+'/frax_deposits.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxWithdraws.head(1000)

In [ ]:
maxActionFraxTimestamp = 0
try:
    maxActionFraxTimestamp = (max(1,oldFraxActions['actions_timestamp'].max()))-1
except:
        maxActionFraxTimestamp = 0
maxActionFraxTimestamp = int(maxActionFraxTimestamp)
maxActionFraxTimestamp

In [ ]:
actionsQuery = frax.Query.actions(
     orderBy='timestamp'
    ,orderDirection='desc'
    ,first=1000
    #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             ,'timestamp_gte': maxBorrowFraxTimestamp
             }
    #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
#{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

)
actions= sg.query_df([
    actionsQuery.id
    ,actionsQuery.type
    ,actionsQuery.amount
    ,actionsQuery.timestamp
    ,actionsQuery.block
    ,actionsQuery.token.name
    ,actionsQuery.root.address
    ,actionsQuery.pair.asset.symbol
    ,actionsQuery.pair.asset.decimals
    ,actionsQuery.pair.collateral.symbol
    ,actionsQuery.pair.collateral.decimals
    ]
)
actions['dateTime'] =(pd.to_datetime(actions['actions_timestamp'],unit='s'))
actions.head(100)

In [ ]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
                 ,'timestamp_gte': maxBorrowFraxTimestamp
                 }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ,actionsQuery.pair.asset.symbol
        ,actionsQuery.pair.asset.decimals
        ,actionsQuery.pair.collateral.symbol
        ,actionsQuery.pair.collateral.decimals
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionListLength<1000: exit =True
    actionTicker = actionTicker+1
actionsDb['DateTime']=(pd.to_datetime(actionsDb['actions_timestamp'],unit='s'))
print("Done. Total Records collected: ", actionsDbLength)

In [ ]:
actionsDb['transactionReadableAmount'] = actionsDb['actions_amount']/(10**(actionsDb['actions_pair_asset_decimals']))
actionsDb['transactionReadableAmount']=actionsDb['transactionReadableAmount'].astype('float')
actionsDb.head(2000)

In [ ]:
fraxActionsDb= pd.DataFrame()
fraxActionsDbTimed =pd.concat([actionsDb, oldFraxActions])
fraxActionsDbTimed.head(1000)

In [ ]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )


In [ ]:
fraxActionsDbTimed.head(10)

In [ ]:
#we might as well create all the DBs here and save ourselves time and grief
fraxBorrowsDbTimed = pd.DataFrame()
fraxBorrowsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'borrowAsset' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxDepositsDbTimed = pd.DataFrame()
fraxDepositsDbTimed =fraxBorrows = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'addCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxWithdrawsDbTimed = pd.DataFrame()
fraxWithdrawsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'removeCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

In [ ]:
fraxBorrowsDbTimed.head(1000)

In [ ]:
fraxDepositsDbTimed.head(1000)

In [ ]:
fraxWithdrawsDbTimed.head(10)

In [ ]:
#fraxActionsDbTimed = pd.concat([oldFraxBorrows,fraxBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
fraxActionsDbTimed.sort_values(['actions_id','actions_type','actions_amount','actions_timestamp','actions_block','actions_token_name','actions_root_address','DateTime'])
fraxActionsDbTimed.drop_duplicates(keep='first' )
fraxActionsDbTimed.head(100)

In [ ]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_actions.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_actions_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

In [ ]:
fraxBorrowsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_borrows.csv'
fraxBorrowsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxBorrowsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

In [ ]:
fraxDepositsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_deposits.csv'
fraxBorrowsDbTimed.to_csv(path, index = False)

path =file+'/frax_deposits_piped.csv'
fraxBorrowsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

In [ ]:
fraxWithdrawsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_borrows.csv'
fraxWithdrawsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxWithdrawsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxWithdrawsDbTimedLength,' records at', (dt.datetime.now()) )

Collateral check

In [ ]:
totalgOHMDepositedFrax = sum(fraxDepositsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMDepositedFrax,2),  'gOHM deposited cumulatively as collateral  to date')

In [ ]:
totalgOHMWithdrawnFrax = sum(fraxWithdrawsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMWithdrawnFrax,2),  'gOHM withdrawn cumulatively as collateral  to date')

In [ ]:
collateralBalance = totalgOHMDepositedFrax-totalgOHMWithdrawnFrax
print(totalgOHMDepositedFrax, ' - ',totalgOHMWithdrawnFrax,'=', collateralBalance, 'currently deposited as collateral in Silo' )

**********************************************************
MERGE DIFFERENT DBs

In [ ]:
#COLLATERALDEPOSITS
fraxDepositsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime"
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol"
    ", transactionReadableAmount ReadableAmount "
    "from fraxDepositsDbTimed "
).df()
fraxDepositsDbPreMerge['protocol'] = 'Frax'
fraxDepositsDbPreMerge['action'] = 'depositCollateral'
fraxDepositsDbPreMerge.head(10)



In [ ]:
siloDepositEventsDbTimedClean.head(1)

In [ ]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


DONT FORGET VESTA WHEN IT GETS FIXED

In [ ]:
gOHMCollateralDeposits = pd.concat([siloDepositsDbPreMerge,fraxDepositsDbPreMerge])
gOHMCollateralDeposits.head(1000)

In [ ]:
gOHMCollateralDepositsLength = len(gOHMCollateralDeposits)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/gOHMCollateralDeposits_piped.csv'
gOHMCollateralDeposits.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralDepositsLength,' records at', (dt.datetime.now()) )

In [ ]:
#ALL WITHDRAW EVENTS

In [ ]:
fraxWithdrawsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime "
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxWithdrawsDbTimed "
).df()
fraxWithdrawsDbPreMerge['protocol'] = 'Frax'
fraxWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
fraxWithdrawsDbPreMerge.head(1000)

In [ ]:
siloWithdrawsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloWithdrawEventsDbTimedClean "
).df()
siloWithdrawsDbPreMerge['protocol'] = 'silo'
siloWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
siloWithdrawsDbPreMerge.head(1000)


DONT FORGET VESTA

In [ ]:
gOHMCollateralWithdrawals = pd.concat([siloWithdrawsDbPreMerge,fraxWithdrawsDbPreMerge])
gOHMCollateralWithdrawals.head(1000)

In [ ]:
gOHMCollateralWithdrawalsLength = len(gOHMCollateralWithdrawals)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False)

path =file+'/gOHMCollateralWithdrawals_piped.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralWithdrawalsLength,' records at', (dt.datetime.now()) )

In [ ]:
#BORROWS

In [ ]:
fraxBorrowsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime as dateTime "
    ", actions_root_address transaction_from "
    ", 'FRAX' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxBorrowsDbTimed "
).df()
fraxBorrowsDbPreMerge['protocol'] = 'Frax'
fraxBorrowsDbPreMerge['action'] = 'borrowStablecoin'
fraxBorrowsDbPreMerge.head(10)

In [ ]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", 'XAI' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['protocol'] = 'silo'
siloBorrowsDbPreMerge['action'] = 'borrowStablecoin'
siloBorrowsDbPreMerge.head(10)

DONT FORGET VESTA

In [ ]:
stablecoinsBorrowedWithGOHMCollateral = pd.concat([siloBorrowsDbPreMerge,fraxBorrowsDbPreMerge])
stablecoinsBorrowedWithGOHMCollateral.head(1000)

In [ ]:
stablecoinsBorrowedWithGOHMCollateralLength = len(stablecoinsBorrowedWithGOHMCollateral)
path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits.csv'
stablecoinsBorrowedWithGOHMCollateral.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv'
stablecoinsBorrowedWithGOHMCollateral.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralLength,' records at', (dt.datetime.now()) )

**********************************************************
MERGE EVERYTHING


In [ ]:
stablecoinsBorrowedWithGOHMCollateral.head(1)

In [ ]:
stablecoinsBorrowedWithGOHMCollateralEvents = pd.DataFrame()
stablecoinsBorrowedWithGOHMCollateralEvents = pd.concat([gOHMCollateralDeposits,gOHMCollateralWithdrawals,stablecoinsBorrowedWithGOHMCollateral])
stablecoinsBorrowedWithGOHMCollateralEvents.head(1000)

In [ ]:
stablecoinsBorrowedWithGOHMCollateralEventsLength = len(stablecoinsBorrowedWithGOHMCollateralEvents)
path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralEventsLength,' records at', (dt.datetime.now()) )

FRAX DAILY SNAPS

In [ ]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.pair.collateral.symbol
    ,pairsQuery.pair.collateral.decimals
    ,pairsQuery.pair.asset.symbol
    ,pairsQuery.pair.asset.decimals
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

In [ ]:
pairSnapshotsDb.head(10)

In [ ]:
pairSnapshotsDb['pairPerDays_totalCollateralReadable'] = pairSnapshotsDb['pairPerDays_totalCollateral']/(10**(pairSnapshotsDb['pairPerDays_pair_collateral_decimals']))
pairSnapshotsDb['pairPerDays_totalCollateralReadable']=pairSnapshotsDb['pairPerDays_totalCollateralReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'] = pairSnapshotsDb['pairPerDays_totalAssetAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalAssetAmountReadable']=pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'] = pairSnapshotsDb['pairPerDays_totalBorrowAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable']=pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'].astype('float')


pairSnapshotsDb['dateTime'] =(pd.to_datetime(pairSnapshotsDb['pairPerDays_timestamp'],unit='s'))

pairSnapshotsDb.drop_duplicates(keep ="first",inplace = True)


pairSnapshotsDb.head(2000)

In [ ]:
fraxPairSnapshotsLength = len(pairSnapshotsDb)
path =file+'/fraxPairSnapshotsLength.csv'
pairSnapshotsDb.to_csv(path, index = False)

path =file+'/fraxPairSnapshotsLength_piped.csv'
pairSnapshotsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxPairSnapshotsLength,' records at', (dt.datetime.now()) )

In [ ]:
fraxCollateral = pairSnapshotsDb.iat[pairSnapshotsTicker,37]
fraxCollateral

In [ ]:
siloCollateralBalance

In [ ]:
totalBorrowedUSDFrax= pairSnapshotsDb.iat[pairSnapshotsTicker,40]
totalBorrowedUSDFrax

In [ ]:
collateralGohmLevels = pd.DataFrame({
    'protocol':['Frax','vesta','silo'],
    'collateralBalance':[fraxCollateral,0,siloCollateralBalance],
    'totalBorrowed':[totalBorrowedUSDFrax,0,totalBorrowedUSDSilo]
})
collateralGohmLevels.head(10)

In [ ]:
collateralGohmLevelsLength = len(collateralGohmLevels)
path =file+'/collateralGohmLevels.csv'
collateralGohmLevels.to_csv(path, index = False)

path =file+'/collateralGohmLevels_piped.csv'
collateralGohmLevels.to_csv(path, index = False, sep='|')
print(path, 'file saved,', collateralGohmLevelsLength,' records at', (dt.datetime.now()) )

In [ ]:
totalBorrowedUSDSilo